In [1]:
import random

In [2]:
def genAlg(numCourses,courses,popSize,times,rooms,maxIters,isElite):
	population = []
	fit = []
	for i in range(0,popSize):
		for j in range(0,numCourses):
			population.append(times[random.randrange(0,len(times))])
			population.append(rooms[random.randrange(0,len(rooms))])
			#Course
			population.append(courses[0+5*j])
			#Professor
			population.append(courses[1+5*j])
			'''#Requested Times
			population.append(courses[2+5*j])
			#Requested Rooms
			population.append(courses[3+5*j])
			#Size
			population.append(courses[4+5*j])'''

	fit = fitness(population,popSize,numCourses,rooms,times)
 
	i = 0
	while (i < maxIters) and ((float(max(fit))/(numCourses*4)) < 0.9):
		#print(i)
		newPop = []
		toPull = fit[:]
		toPull.sort()
		toPull = toPull[(len(toPull)-int(0.1*len(toPull))):len(toPull)]
		topIndexs = []
		tempFit = fit[:]
		tempPop = []
		for j in range(0,len(toPull)):
			topIndexs.append(tempFit.index(toPull[j]))
			tempFit[topIndexs[j]] = 0
			tempPop += population[topIndexs[j]*4*numCourses:(topIndexs[j]*4*numCourses)+4*numCourses]

		for j in range(0,popSize):
			if isElite:
				howToReproduce = random.random()
				r = random.randrange(0,len(topIndexs))
				winner = tempPop[(r*numCourses*4):((r*numCourses*4)+(numCourses*4))]
				child = 0

				if howToReproduce < 0.15:
					child = mutate(winner,numCourses,times,rooms)
				else:
					howToReproduce = random.randrange(0,3)
					if howToReproduce == 0:
						child = swapRooms(winner,numCourses,rooms)
					elif howToReproduce == 1:
						child = swapTimes(winner,numCourses,times)
					else:
						child = changeCourseTime(winner,numCourses,times)

				if fitness(child,1,numCourses,rooms,times)[0] > fitness(winner,1,numCourses,rooms,times)[0]:
					newPop += child
				else:
					newPop += winner
			else:
				pick1 = random.randrange(0,popSize)
				pick2 = random.randrange(0,popSize)

				winner = 0
				if fit[pick1] > fit[pick2]:
					winner = pick1
				else:
					winner = pick2

				winner = population[(winner*numCourses*4):((winner*numCourses*4)+(numCourses*4))]

				howToReproduce = random.random()
				child = 0

				if howToReproduce < 0.15:
					child = mutate(winner,numCourses,times,rooms)
				else:
					howToReproduce = random.randrange(0,3)
					if howToReproduce == 0:
						child = swapRooms(winner,numCourses,rooms)
					elif howToReproduce == 1:
						child = swapTimes(winner,numCourses,times)
					else:
						child = changeCourseTime(winner,numCourses,times)

				if fitness(child,1,numCourses,rooms,times)[0] > fitness(winner,1,numCourses,rooms,times)[0]:
					newPop += child
				else:
					newPop += winner

		population = newPop[:]
		fit = fitness(population,popSize,numCourses,rooms,times)
		#sfit = fit[:]
		#sfit.sort()
		#file = open("maxs.txt",'a')
		#file.write(str(float(max(fit))/(numCourses*5))+'\n')
		#file.close
		#file = open("pop.txt",'a')
		#file.write(str(float(sum(fit))/(popSize*numCourses*5))+'\n')
		#file.close
		#print "Fitness: " + str(sum(fit)/(popSize*numCourses*5))
		i += 1

	maxIndex = fit.index(max(fit))
	return population[4*maxIndex:4*maxIndex+4*numCourses]

In [3]:
def swapRooms(schedule,numCourses,rooms):
  sched = schedule[:]
  room1 = rooms[random.randrange(0,len(rooms))]
  room2 = rooms[random.randrange(0,len(rooms))]
  for i in range(0,len(sched),4):
    if sched[i+1] == room1:
      sched[i+1] = room2
    elif sched[i+1] == room2:
      sched[i+1] = room1

  return sched

In [4]:
def swapTimes(schedule,numCourses,times):
  sched = schedule[:]
  time1 = times[random.randrange(0,len(times))]
  time2 = times[random.randrange(0,len(times))]
  for i in range(0,len(sched),7):
    if sched[i] == time1:
      sched[i] = time2 
    elif sched[i] == time2:
      sched[i] = time1 

  return sched

In [5]:
def changeCourseTime(schedule,numCourses,times):
  sched = schedule[:]
  time = times[random.randrange(0,len(times))]
  course= random.randrange(0,numCourses)

  sched[course*4] = time

  return sched

In [8]:
def mutate(schedule,numCourses,times,rooms):
  sched = schedule[:]
  time = times[random.randrange(0,len(times))]
  room = rooms[random.randrange(0,len(rooms))]
  course= random.randrange(0,numCourses)

  sched[course*4] = time
  sched[course*4+1] = room 

  return sched

In [24]:
def fitness(pop,popSize,numCourses,rooms,times):

	fit = []

	for i in range(0,popSize):
		afit = 0
		curChrom = i*numCourses*4
		for j in range(0,numCourses*4,4):
			sameTimeSameRoom = 5
			sameTimeSameProf = 8
			#requestedRoom = 0
			#requestedTime = 0
			#roomSizeNeed = 1
			tempTime = pop[curChrom+j]
			tempRoom = pop[curChrom+j+1]
			tempProf = pop[curChrom+j+3]
			
			

			for k in range(j+4,numCourses*4,4):
				if (tempTime == pop[curChrom+k]) and (tempRoom == pop[curChrom+k+1]):
					sameTimeSameRoom = 0
				if (tempTime == pop[curChrom+k]) and (tempProf == pop[curChrom+k+3]):
					sameTimeSameProf = 0

			afit += sameTimeSameRoom + sameTimeSameProf #+ requestedRoom + requestedTime + roomSizeNeed     
		fit.append(afit)
	return fit


# Pruebas

In [9]:
from itertools import combinations
import numpy as np
import pandas as pd

In [11]:
equipos1=pd.read_csv('equipos.csv',delimiter=';',header=None)
equipos1.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1a,1b,1c,1d,1e,1f,1g,1h,1i,1j,1k,1l,1m,1n,1o,1p
1,2a,2b,2c,2d,2e,2f,2g,2h,2i,2j,2k,2l,2m,2n,2o,2p
2,3a,3b,3c,3d,3e,3f,3g,3h,3i,3j,3k,3l,3m,3n,3o,3p
3,4a,4b,4c,4d,4e,4f,4g,4h,4i,4j,4k,4l,4m,4n,4o,4p
4,5a,5b,5c,5d,5e,5f,5g,5h,5i,5j,5k,5l,5m,5n,5o,5p
5,6a,6b,6c,6d,6e,6f,6g,6h,6i,6j,6k,6l,6m,6n,6o,6p
6,7a,7b,7c,7d,7e,7f,7g,7h,7i,7j,7k,7l,7m,7n,7o,7p
7,8a,8b,8c,8d,8e,8f,8g,8h,8i,8j,8k,8l,8m,8n,8o,8p
8,9a,9b,9c,9d,9e,9f,9g,9h,9i,9j,9k,9l,9m,9n,9o,9p
9,10a,10b,10c,10d,10e,10f,10g,10h,10i,10j,10k,10l,10m,10n,10o,10p


Tabla de grupos, 12 equipos por grupos, 16 grupos

In [13]:
timez=[]
for i in range(30):
  timez.append(i+1)


In [14]:
def combinar_partidos(equipos1,group):
  equipos=equipos1.to_numpy()
  partidos_grupo= combinations(equipos[group],2)
  grupo=[]
  for i in list(partidos_grupo):
    grupo.append(str(i))
  return grupo

In [51]:
grupos=[]
for i in range(10):
  GRUPO=combinar_partidos(equipos1,i)
  grupos.extend(GRUPO)

len(grupos)

660

In [25]:
# JUST FOR TESTING
def test(timez,grupo):
	courses = grupo
	profs = [1,2,3,4,5,6,7,8]
	times = timez
	rooms = [1,2,3,4,5]

	initial = []

	for i in range(0,len(grupo)):
		initial.append(courses[i])
		initial.append(profs[random.randrange(0,8)])
		temp = []
		temp.append(times[random.randrange(0,8)])
		temp.append(times[random.randrange(0,8)])
		initial.append(temp)
		#initial.append(times[random.randrange(0,8)])
		temp2 = []
		temp2.append(rooms[random.randrange(0,5)])
		temp2.append(rooms[random.randrange(0,5)])
		initial.append(temp2)
		#initial.append(rooms[random.randrange(0,5)])
		initial.append(20)

	#printCourse(genAlg(len(grupo),initial,200,times,rooms,1000,True))
	schedule=genAlg(len(grupo),initial,200,times,rooms,1000,True)

	return schedule

In [52]:
fixture1=[]
fixture1=test(timez,grupos)

fixture=np.array(fixture1)
fixture=np.reshape(fixture,(len(grupos),4))
tablas=pd.DataFrame(fixture,columns=['Periodo','Cancha','Partido','Día'])
tablas

,Periodo,Cancha,Partido,Día
0,9,5,"('1c', '8c')",5
1,20,4,"('1c', '9c')",5
2,29,5,"('1c', '10c')",7
3,8,1,"('1c', '11c')",2
4,17,2,"('1c', '12c')",4
...,...,...,...,...
655,14,5,"('1c', '3c')",8
656,21,3,"('1c', '4c')",2
657,24,4,"('1c', '5c')",3
658,24,3,"('1c', '6c')",1


In [57]:
def horarios_cancha1(programa):
  c=0
  k=1
  for i in programa[:,1]:
    if i == '1':
     #programa=np.delete(programa,c,axis=0)
     print(programa[c],k)
     k+=1
    c=c+1
    

In [58]:
#tabla_cancha1=horarios_cancha1(fixture)
#tablas=pd.DataFrame(tabla_cancha1)
#tablas

horarios_cancha1(fixture)

['8' '1' "('1c', '11c')" '2'] 1
['29' '1' "('2c', '7c')" '6'] 2
['14' '1' "('2c', '10c')" '2'] 3
['14' '1' "('5c', '10c')" '7'] 4
['26' '1' "('5c', '12c')" '2'] 5
['13' '1' "('6c', '10c')" '7'] 6
['25' '1' "('6c', '11c')" '8'] 7
['13' '1' "('7c', '10c')" '2'] 8
['24' '1' "('8c', '12c')" '1'] 9
['28' '1' "('1d', '4d')" '5'] 10
['3' '1' "('1d', '8d')" '8'] 11
['21' '1' "('1d', '9d')" '4'] 12
['20' '1' "('1d', '11d')" '4'] 13
['14' '1' "('2d', '3d')" '6'] 14
['10' '1' "('2d', '4d')" '5'] 15
['8' '1' "('2d', '6d')" '7'] 16
['14' '1' "('4d', '12d')" '2'] 17
['3' '1' "('5d', '9d')" '2'] 18
['21' '1' "('5d', '12d')" '6'] 19
['12' '1' "('7d', '10d')" '3'] 20
['16' '1' "('8d', '9d')" '2'] 21
['18' '1' "('8d', '10d')" '6'] 22
['1' '1' "('1e', '6e')" '7'] 23
['22' '1' "('1e', '8e')" '8'] 24
['8' '1' "('1e', '11e')" '3'] 25
['16' '1' "('1e', '12e')" '3'] 26
['22' '1' "('3e', '6e')" '5'] 27
['28' '1' "('3e', '9e')" '8'] 28
['27' '1' "('3e', '10e')" '2'] 29
['3' '1' "('4e', '6e')" '3'] 30
['12' '1' 